In [ ]:
# Load necessary libraries
library(dplyr)
library(Matrix)
library(data.table)
library(Seurat)
library(ggplot2)
library(RColorBrewer)
library(cowplot)
library(tidyverse)
library(tidyr)
library(viridis)
library(harmony)
library(MASS) #
library(scales)
library(AnnotationHub)	
library(org.Hs.eg.db)  
library(clusterProfiler)
library(biomaRt)
library(ggthemes)
library(msigdbr)
library(conflicted)
library(stringr)


In [ ]:
# Define chip and area lists
chiplist = c("CON1.1","CON1.2","CON2.1","CON2.2","CON3.1","CON3.2", "CON4.1","CON4.2",
             "AD1.1","AD1.2","AD2.1","AD2.2","AD3.1","AD3.2","AD4.1","AD4.2")
arealist = c('DG','SLRM','FAS','CA1','CA2','CA3','CA4')

# Initialize an empty data frame
merge_cellbin_melt = data.frame()

# Loop over chips and areas to calculate density
for (chips in chiplist) {
    for (area in arealist) {
        bin_num = bin[chips, area]
        subarea = data[data$area == area,]
        cellbin <- table(subarea[,"Chips"], subarea[,"area"])  
        cellbin <- as.data.frame.matrix(cellbin)
        cellbin$Chips <- rownames(cellbin)
        cellbin_melt <- melt(cellbin)
        cellbin_melt <- cellbin_melt[cellbin_melt$Chips == chips,]   
        cellbin_melt$density <- cellbin_melt$value / (bin_num * 0.0025)
        merge_cellbin_melt <- rbind(cellbin_melt, merge_cellbin_melt)
    }
}

# Add a new column 'Class' to indicate 'AD' or 'CON'
merge_cellbin_melt$Class = gsub("(AD|CON).*", "\\1", merge_cellbin_melt$Chips)
merge_cellbin_melt$Class <- factor(merge_cellbin_melt$Class, levels = c("CON", "AD"))

# Display the first few rows of the merged data
head(merge_cellbin_melt)

# Initialize an empty data frame for the merged data
mergedata = data.frame()

# Loop over 'AD' and 'CON' classes to calculate mean, standard deviation, and standard error
for (i in c("AD", "CON")) {
    subdata = subset(merge_cellbin_melt, Class == i)
    data1 <- subdata %>% 
      group_by(variable) %>%
      summarize(
        Mean = mean(density),
        StdDev = sd(density),
        StdErr = StdDev / sqrt(n())
      )
    data1$Class = i
    mergedata <- rbind(data1, mergedata)
}

# Rename the first column to 'area'
colnames(mergedata)[1] = "area"

# Set plot dimensions
options(repr.plot.width = 11, repr.plot.height = 4) 

# Define colors for the plot
cols = c("#8dc7c2", "#e94c5f")

# Order factors for 'area' and 'Class'
mergedata$area = factor(mergedata$area, levels = c('CA1', 'CA2', 'CA3', 'CA4', 'DG', 'FAS', 'SLRM'))
mergedata$Class = factor(mergedata$Class, levels = c('CON', 'AD'))

# Plot the data
ggplot(mergedata, aes(x = area, y = Mean, colour = Class, group = Class, shape = Class)) +
  geom_line(aes(color = Class), size = 0.5) + 
  geom_errorbar(aes(ymin = Mean - StdErr, ymax = Mean + StdErr), width = 0.1, size = 0.7) +
  geom_point(size = 5) + 
  scale_color_manual(values = cols) + 
  labs(title = '', x = "", y = "\n \n \n ") +
  theme_classic() +
  labs(title = paste0(""), x = "area", y = expression("Density"))
